In [1]:
import plotly.graph_objects as go
import numpy as np

class Painter:

    def __init__(self, bins):
        self.items = bins.items
        self.width = bins.width
        self.height = bins.height
        self.depth = bins.depth

    def _plotCube(self, fig, x, y, z, dx, dy, dz, color='red', text="", alpha=0.5):
        """ Auxiliary function to plot a cube with solid colors. """
        vertices = [
            [x, y, z], [x + dx, y, z], [x + dx, y + dy, z], [x, y + dy, z],
            [x, y, z + dz], [x + dx, y, z + dz], [x + dx, y + dy, z + dz], [x, y + dy, z + dz]
        ]
        # Define the 12 triangles composing the cube
        faces = [
            [0, 1, 2, 3], [4, 5, 6, 7], [0, 1, 5, 4], 
            [2, 3, 7, 6], [1, 2, 6, 5], [0, 3, 7, 4]
        ]

        x_data = [vertices[face[i]][0] for face in faces for i in range(4)]
        y_data = [vertices[face[i]][1] for face in faces for i in range(4)]
        z_data = [vertices[face[i]][2] for face in faces for i in range(4)]

        fig.add_trace(go.Mesh3d(
            x=x_data, y=y_data, z=z_data,
            i=[0, 4, 8, 12, 16, 20], 
            j=[1, 5, 9, 13, 17, 21], 
            k=[2, 6, 10, 14, 18, 22],
            color=color, opacity=alpha, flatshading=True,
            hovertext=text
        ))

    def _plotCylinder(self, fig, x, y, z, dx, dy, dz, color='red', text="", alpha=0.5):
        """ Auxiliary function to plot a cylinder. """
        theta = np.linspace(0, 2 * np.pi, 30)
        x_circle = dx / 2 * np.cos(theta) + x + dx / 2
        y_circle = dy / 2 * np.sin(theta) + y + dy / 2

        z1 = np.full_like(theta, z)
        z2 = np.full_like(theta, z + dz)

        fig.add_trace(go.Scatter3d(x=x_circle, y=y_circle, z=z1, mode='lines', line=dict(color=color), opacity=alpha))
        fig.add_trace(go.Scatter3d(x=x_circle, y=y_circle, z=z2, mode='lines', line=dict(color=color), opacity=alpha))

        for i in range(len(theta)):
            fig.add_trace(go.Scatter3d(
                x=[x_circle[i], x_circle[i]], y=[y_circle[i], y_circle[i]], z=[z1[i], z2[i]], mode='lines',
                line=dict(color=color), opacity=alpha
            ))

        z_grid, theta_grid = np.meshgrid(np.linspace(z, z + dz, 10), theta)
        x_grid = dx / 2 * np.cos(theta_grid) + x + dx / 2
        y_grid = dy / 2 * np.sin(theta_grid) + y + dy / 2

        fig.add_trace(go.Surface(x=x_grid, y=y_grid, z=z_grid, colorscale=[[0, color], [1, color]], showscale=False, opacity=alpha))

    def plotBoxAndItems(self, title="", alpha=0.2, write_num=False, fontsize=10):
        """ Side effect: Plot the bin and the items it contains. """
        fig = go.Figure()

        # plot bin 
        self._plotCube(fig, 0, 0, 0, float(self.width), float(self.height), float(self.depth), color='black', alpha=0.1)

        # fit rotation type
        for item in self.items:
            x, y, z = item.position
            w, h, d = item.getDimension()
            color = item.color
            text = item.partno if write_num else ""

            if item.typeof == 'cube':
                # plot item of cube
                self._plotCube(fig, float(x), float(y), float(z), float(w), float(h), float(d), color=color, text=text, alpha=alpha)
            elif item.typeof == 'cylinder':
                # plot item of cylinder
                self._plotCylinder(fig, float(x), float(y), float(z), float(w), float(h), float(d), color=color, text=text, alpha=alpha)

        fig.update_layout(title=title, scene_aspectmode='data')
        return fig


In [2]:
from py3dbp import Packer, Bin, Item
'''

If you have multiple boxes, you can change distribute_items to achieve different packaging purposes.
1. distribute_items=True , put the items into the box in order, if the box is full, the remaining items will continue to be loaded into the next box until all the boxes are full  or all the items are packed.
2. distribute_items=False, compare the packaging of all boxes, that is to say, each box packs all items, not the remaining items.

'''

# init packing function
packer = Packer()
#  init bin 
box = Bin('example7-Bin1', (5, 5, 5), 100,0,0)
box2 = Bin('example7-Bin2', (3, 3, 5), 100,0,0)
#  add item
# Item('item partno', (W,H,D), Weight, Packing Priority level, load bear, Upside down or not , 'item color')
packer.addBin(box)
packer.addBin(box2)

packer.addItem(Item(partno='Box-1', name='test1', typeof='cube', WHD=(5, 4, 1), weight=1, level=1,loadbear=100, updown=True, color='yellow'))
packer.addItem(Item(partno='Box-2', name='test2', typeof='cube', WHD=(1, 2, 4), weight=1, level=1,loadbear=100, updown=True, color='olive'))
packer.addItem(Item(partno='Box-3', name='test3', typeof='cube', WHD=(1, 2, 3), weight=1, level=1,loadbear=100, updown=True, color='olive'))
packer.addItem(Item(partno='Box-4', name='test4', typeof='cube', WHD=(1, 2, 2), weight=1, level=1,loadbear=100, updown=True, color='olive'))
packer.addItem(Item(partno='Box-5', name='test5', typeof='cube', WHD=(1, 2, 3), weight=1, level=1,loadbear=100, updown=True, color='olive'))
packer.addItem(Item(partno='Box-6', name='test6', typeof='cube', WHD=(1, 2, 4), weight=1, level=1,loadbear=100, updown=True, color='olive'))
packer.addItem(Item(partno='Box-7', name='test7', typeof='cube', WHD=(1, 2, 2), weight=1, level=1,loadbear=100, updown=True, color='olive'))
packer.addItem(Item(partno='Box-8', name='test8', typeof='cube', WHD=(1, 2, 3), weight=1, level=1,loadbear=100, updown=True, color='olive'))
packer.addItem(Item(partno='Box-9', name='test9', typeof='cube', WHD=(1, 2, 4), weight=1, level=1,loadbear=100, updown=True, color='olive'))
packer.addItem(Item(partno='Box-10', name='test10', typeof='cube', WHD=(1, 2, 3), weight=1, level=1,loadbear=100, updown=True, color='olive'))
packer.addItem(Item(partno='Box-11', name='test11', typeof='cube', WHD=(1, 2, 2), weight=1, level=1,loadbear=100, updown=True, color='olive'))
packer.addItem(Item(partno='Box-12', name='test12', typeof='cube', WHD=(5, 4, 1), weight=1, level=1,loadbear=100, updown=True, color='pink'))
packer.addItem(Item(partno='Box-13', name='test13', typeof='cube', WHD=(1, 1, 4), weight=1, level=1,loadbear=100, updown=True, color='olive'))
packer.addItem(Item(partno='Box-14', name='test14', typeof='cube', WHD=(1, 2, 1), weight=1, level=1,loadbear=100, updown=True, color='pink'))
packer.addItem(Item(partno='Box-15', name='test15', typeof='cube', WHD=(1, 2, 1), weight=1, level=1,loadbear=100, updown=True, color='pink'))
packer.addItem(Item(partno='Box-16', name='test16', typeof='cube', WHD=(1, 1, 4), weight=1, level=1,loadbear=100, updown=True, color='olive'))
packer.addItem(Item(partno='Box-17', name='test17', typeof='cube', WHD=(1, 1, 4), weight=1, level=1,loadbear=100, updown=True, color='olive'))
packer.addItem(Item(partno='Box-18', name='test18', typeof='cube', WHD=(5, 4, 2), weight=1, level=1,loadbear=100, updown=True, color='brown'))

# calculate packing 
packer.pack(
    bigger_first=True,
    # Change distribute_items=False to compare the packing situation in multiple boxes of different capacities.
    distribute_items=False,
    fix_point=True,
    check_stable=True,
    support_surface_ratio=0.75,
    number_of_decimals=0
)

# put order
packer.putOrder()

In [5]:
painter=Painter(box)
painter.plotBoxAndItems(title="test", alpha=0.2, write_num=False, fontsize=10)

In [18]:
import plotly.graph_objects as go
import numpy as np

class Painter:

    def __init__(self, bins):
        self.items = bins.items
        self.width = bins.width
        self.height = bins.height
        self.depth = bins.depth

    def _plotCube(self, fig, x, y, z, dx, dy, dz, color='red', text="", alpha=1):
        vertices = [
            [x, y, z], [x + dx, y, z], [x + dx, y + dy, z], [x, y + dy, z],
            [x, y, z + dz], [x + dx, y, z + dz], [x + dx, y + dy, z + dz], [x, y + dy, z + dz]
        ]
        faces = [
            [0, 1, 2], [0, 2, 3], [4, 5, 6], [4, 6, 7],
            [0, 1, 5], [0, 5, 4], [2, 3, 7], [2, 7, 6],
            [1, 2, 6], [1, 6, 5], [0, 3, 7], [0, 7, 4]
        ]

        x_data = [vertices[face[i]][0] for face in faces for i in range(3)]
        y_data = [vertices[face[i]][1] for face in faces for i in range(3)]
        z_data = [vertices[face[i]][2] for face in faces for i in range(3)]

        fig.add_trace(go.Mesh3d(
            x=x_data, y=y_data, z=z_data,
            color=color, opacity=alpha,
            alphahull=0, hovertext=text
        ))

    def _plotCylinder(self, fig, x, y, z, dx, dy, dz, color='red', text="", alpha=1):
        theta = np.linspace(0, 2 * np.pi, 30)
        x_circle = dx / 2 * np.cos(theta) + x + dx / 2
        y_circle = dy / 2 * np.sin(theta) + y + dy / 2

        z1 = np.full_like(theta, z)
        z2 = np.full_like(theta, z + dz)

        fig.add_trace(go.Scatter3d(x=x_circle, y=y_circle, z=z1, mode='lines', line=dict(color=color), opacity=alpha))
        fig.add_trace(go.Scatter3d(x=x_circle, y=y_circle, z=z2, mode='lines', line=dict(color=color), opacity=alpha))

        for i in range(len(theta)):
            fig.add_trace(go.Scatter3d(
                x=[x_circle[i], x_circle[i]], y=[y_circle[i], y_circle[i]], z=[z1[i], z2[i]], mode='lines',
                line=dict(color=color), opacity=alpha
            ))

        z_grid, theta_grid = np.meshgrid(np.linspace(z, z + dz, 10), theta)
        x_grid = dx / 2 * np.cos(theta_grid) + x + dx / 2
        y_grid = dy / 2 * np.sin(theta_grid) + y + dy / 2

        fig.add_trace(go.Surface(x=x_grid, y=y_grid, z=z_grid, surfacecolor=[color]*len(z_grid), showscale=False, opacity=alpha))

    def plotBoxAndItems(self, title="", alpha=1, write_num=False, fontsize=10):
        fig = go.Figure()

        # plot bin 
        self._plotCube(fig, 0, 0, 0, float(self.width), float(self.height), float(self.depth), color='black', alpha=alpha)

        # fit rotation type
        for item in self.items:
            x, y, z = item.position
            w, h, d = item.getDimension()
            color = item.color
            text = item.partno if write_num else ""

            if item.typeof == 'cube':
                # plot item of cube
                self._plotCube(fig, float(x), float(y), float(z), float(w), float(h), float(d), color=color, text=text, alpha=alpha)
            elif item.typeof == 'cylinder':
                # plot item of cylinder
                self._plotCylinder(fig, float(x), float(y), float(z), float(w), float(h), float(d), color=color, text=text, alpha=alpha)

        fig.update_layout(title=title, scene_aspectmode='data')
        return fig


In [19]:
# Evergreen Real Container (20ft Steel Dry Cargo Container)
# Unit cm/kg
box = Bin(
    partno='example4',
    WHD=(589.8, 243.8, 259.1),
    max_weight=28080,
    corner=15,
    put_type=0
)

packer.addBin(box)

# Dyson DC34 (170 * 82 * 46 , 85.12 kg)
# 15 pcs per case, 170 * 82 * 46 (85.12 kg)
for i in range(15): 
    packer.addItem(Item(
        partno='Dyson DC34 Animal{}'.format(str(i + 1)),
        name='Dyson',
        typeof='cube',
        WHD=(170, 82, 46),
        weight=85.12,
        level=1,
        loadbear=100,
        updown=True,
        color='#FF0000'
    ))

# Washing Machine (85 * 60 * 60 , 10 kg)
# 18 pcs per case, 85 * 60 * 60 (10 kg)
for i in range(18):
    packer.addItem(Item(
        partno='wash{}'.format(str(i + 1)),
        name='wash',
        typeof='cube',
        WHD=(85, 60, 60),
        weight=10,
        level=1,
        loadbear=100,
        updown=True,
        color='#FFFF37'
    ))

# 42U Standard Cabinet (60 * 80 * 200 , 80 kg)
# 15 pcs per case, 60 * 80 * 200 (80 kg)
for i in range(15):
    packer.addItem(Item(
        partno='Cabinet{}'.format(str(i + 1)),
        name='cabint',
        typeof='cube',
        WHD=(60, 80, 200),
        weight=80,
        level=1,
        loadbear=100,
        updown=True,
        color='#842B00'
    ))

# Server (70 * 100 * 30 , 20 kg)
# 42 pcs per case, 70 * 100 * 30 (20 kg)
for i in range(42):
    packer.addItem(Item(
        partno='Server{}'.format(str(i + 1)),
        name='server',
        typeof='cube',
        WHD=(70, 100, 30),
        weight=20,
        level=1,
        loadbear=100,
        updown=True,
        color='#0000E3'
    ))

# Cylinder (50-gallon Oil Drum, 80 cm diameter, 120 cm height, 170 kg)
# 20 pcs per case, 80 cm diameter, 120 cm height (170 kg)
for i in range(20):
    packer.addItem(Item(
        partno='Cylinder{}'.format(str(i + 1)),
        name='cylinder',
        typeof='cylinder',
        WHD=(80, 120, 80),  # Note: WHD for a cylinder is unconventional; it represents diameter and height
        weight=170,
        level=1,
        loadbear=50,
        updown=False,
        color='#00FF00'
    ))

# calculate packing 
packer.pack(
    bigger_first=True,
    # Change distribute_items=False to compare the packing situation in multiple boxes of different capacities.
    distribute_items=False,
    fix_point=True,
    check_stable=True,
    support_surface_ratio=0.75,
    number_of_decimals=0
)

# put order
packer.putOrder()

In [30]:
painter=Painter(box)
painter.plotBoxAndItems(title="test", alpha=0.2, write_num=False, fontsize=10)

In [1]:
from py3dbp import Packer, Bin, Item #, Painter
import time
# start = time.time()

'''

This example can be used to compare the fix_point function with and without the fix_point function.

'''

# init packing function
packer = Packer()

# Evergreen Real Container (20ft Steel Dry Cargo Container)
# Unit cm/kg
box = Bin(
    partno='example0',
    WHD=(589.8,243.8,259.1),
    # WHD=(600,280,269),    
    max_weight=28080,
    corner=15,
    put_type=0
)

packer.addBin(box)

# dyson DC34 (20.5 * 11.5 * 32.2 ,1.33kg)
# 64 pcs per case ,  82 * 46 * 170 (85.12)
for i in range(5): 
    packer.addItem(Item(
        partno='Dyson DC34 Animal{}'.format(str(i+1)),
        name='Dyson', 
        typeof='cube',
        WHD=(170, 82, 46), 
        weight=85.12,
        level=1,
        loadbear=500,
        updown=True,
        color='#FF0000')
    )

# washing machine (85 * 60 *60 ,10 kG)
# 1 pcs per case, 85 * 60 *60 (10)
for i in range(10):
    packer.addItem(Item(
        partno='wash{}'.format(str(i+1)),
        name='wash',
        typeof='cube',
        WHD=(85, 60, 60), 
        weight=10,
        level=1,
        loadbear=500,
        updown=True,
        color='#FFFF37'
    ))

# 42U standard cabinet (60 * 80 * 200 , 80 kg)
# one per box, 60 * 80 * 200 (80)
for i in range(5):
    packer.addItem(Item(
        partno='Cabinet{}'.format(str(i+1)),
        name='cabint',
        typeof='cube',
        WHD=(60, 80, 200), 
        weight=80,
        level=1,
        loadbear=500,
        updown=True,
        color='#842B00')
    )

# Server (70 * 100 * 30 , 20 kg) 
# one per box , 70 * 100 * 30 (20)
for i in range(10):
    packer.addItem(Item(
        partno='Server{}'.format(str(i+1)),
        name='server',
        typeof='cube',
        WHD=(70, 100, 30), 
        weight=20,
        level=1,
        loadbear=500,
        updown=True,
        color='#0000E3')
    )


# calculate packing
packer.pack(
    bigger_first=True,
    distribute_items=False,
    check_stable=True,
    fix_point=True, # Try switching fix_point=True/False to compare the results
    number_of_decimals=0
)

# # print result
# for box in packer.bins:

#     volume = box.width * box.height * box.depth
#     print(":::::::::::", box.string())

#     print("FITTED ITEMS:")
#     volume_t = 0
#     volume_f = 0
#     unfitted_name = ''

#     # '''
#     for item in box.items:
#         print("partno : ",item.partno)
#         print("type : ",item.name)
#         print("color : ",item.color)
#         print("position : ",item.position)
#         print("rotation type : ",item.rotation_type)
#         print("W*H*D : ",str(item.width) +'*'+ str(item.height) +'*'+ str(item.depth))
#         print("volume : ",float(item.width) * float(item.height) * float(item.depth))
#         print("weight : ",float(item.weight))
#         volume_t += float(item.width) * float(item.height) * float(item.depth)
#         print("***************************************************")
#     print("***************************************************")
#     # '''
#     print("UNFITTED ITEMS:")
#     for item in box.unfitted_items:
#         print("partno : ",item.partno)
#         print("type : ",item.name)
#         print("color : ",item.color)
#         print("W*H*D : ",str(item.width) +'*'+ str(item.height) +'*'+ str(item.depth))
#         print("volume : ",float(item.width) * float(item.height) * float(item.depth))
#         print("weight : ",float(item.weight))
#         volume_f += float(item.width) * float(item.height) * float(item.depth)
#         unfitted_name += '{},'.format(item.partno)
#         print("***************************************************")
#     print("***************************************************")
#     print('space utilization : {}%'.format(round(volume_t / float(volume) * 100 ,2)))
#     print('residual volumn : ', float(volume) - volume_t )
#     print('unpack item : ',unfitted_name)
#     print('unpack item volumn : ',volume_f)
#     print("gravity distribution : ",box.gravity)
#     # '''
#     stop = time.time()
#     print('used time : ',stop - start)

#     # draw results
#     print(box)
#     painter = Painter(box)
#     fig = painter.plotBoxAndItems(
#         title=box.partno,
#         alpha=0.2,
#         write_num=True,
#         fontsize=10
#     )
# fig.show()


In [3]:
results = []
    
for box in packer.bins:
    volume = box.width * box.height * box.depth
    volume_t = 0
    volume_f = 0
    unfitted_name = ''

    for item in box.items:
        volume_t += float(item.width) * float(item.height) * float(item.depth)

    for item in box.unfitted_items:
        volume_f += float(item.width) * float(item.height) * float(item.depth)
        unfitted_name += '{},'.format(item.partno)

    box_stats = {
        'space_utilization': round(volume_t / float(volume) * 100, 2),
        'residual_volume': float(volume) - volume_t,
        'unfitted_items': unfitted_name.strip(','),
        'unfitted_volume': volume_f,
        'gravity_distribution': box.gravity
    }

    results.append(box_stats)
    
print(results)

[{'space_utilization': 19.83, 'residual_volume': 29893680.0, 'unfitted_items': 'Dyson DC34 Animal5,wash1,wash2,wash3,wash4,wash5,wash6,wash7,wash8,wash9,wash10,Server1,Server2,Server3,Server4,Server5,Server6,Server7,Server8,Server9,Server10', 'unfitted_volume': 5801240.0, 'gravity_distribution': [56.58, 23.15, 17.07, 3.2]}]


In [4]:
box._plot()

In [5]:
packer = Packer()
#  init bin 
box = Bin('example6', (5, 4, 7), 100,0,0)
#  add item
# Item('item partno', (W,H,D), Weight, Packing Priority level, load bear, Upside down or not , 'item color')
packer.addBin(box)
packer.addItem(Item(partno='Box-1', name='test', typeof='cube', WHD=(5, 4, 1), weight=1, level=1,loadbear=100, updown=True, color='yellow'))
packer.addItem(Item(partno='Box-2', name='test', typeof='cube', WHD=(1, 1, 4), weight=1, level=2,loadbear=100, updown=True, color='olive'))
packer.addItem(Item(partno='Box-3', name='test', typeof='cube', WHD=(3, 4, 2), weight=1, level=3,loadbear=100, updown=True, color='pink'))
packer.addItem(Item(partno='Box-4', name='test', typeof='cube', WHD=(1, 1, 4), weight=1, level=4,loadbear=100, updown=True, color='olive'))
packer.addItem(Item(partno='Box-5', name='test', typeof='cube', WHD=(1, 2, 1), weight=1, level=5,loadbear=100, updown=True, color='pink'))
packer.addItem(Item(partno='Box-6', name='test', typeof='cube', WHD=(1, 2, 1), weight=1, level=6,loadbear=100, updown=True, color='pink'))
packer.addItem(Item(partno='Box-7', name='test', typeof='cube', WHD=(1, 1, 4), weight=1, level=7,loadbear=100, updown=True, color='olive'))
packer.addItem(Item(partno='Box-8', name='test', typeof='cube', WHD=(1, 1, 4), weight=1, level=8,loadbear=100, updown=True, color='olive'))# Try switching WHD=(1, 1, 3) and (1, 1, 4) to compare the results
packer.addItem(Item(partno='Box-9', name='test', typeof='cube', WHD=(5, 4, 2), weight=1, level=9,loadbear=100, updown=True, color='brown'))

# calculate packing 
packer.pack(
    bigger_first=True,
    distribute_items=False,
    fix_point=True,
    check_stable=True,
    support_surface_ratio=0.75,
    number_of_decimals=0
)

# put order
packer.putOrder()


In [6]:
box._plot()

In [7]:
# init packing function
packer = Packer()
#  init bin 
box = Bin('example3', (6, 1, 5), 100,0,put_type=0)
#  add item
# Item('item partno', (W,H,D), Weight, Packing Priority level, load bear, Upside down or not , 'item color')
packer.addBin(box)
# If all item WHD=(2, 1, 3) , item can be fully packed into box, but if choose one item and modify WHD=(3, 1, 2) , item can't be fully packed into box.
packer.addItem(Item(partno='Box-1',name='test',typeof='cube', WHD=(2, 1, 3), weight=1, level=1,loadbear=100, updown=True, color='yellow'))
packer.addItem(Item(partno='Box-2',name='test',typeof='cube', WHD=(3, 1, 2), weight=1, level=1,loadbear=100, updown=True, color='pink')) # Try switching WHD=(3, 1, 2) and (2, 1, 3) to compare the results
packer.addItem(Item(partno='Box-3',name='test',typeof='cube', WHD=(2, 1, 3), weight=1,level= 1,loadbear=100, updown=True, color='brown'))
packer.addItem(Item(partno='Box-4',name='test',typeof='cube', WHD=(2, 1, 3), weight=1, level=1,loadbear=100, updown=True, color='cyan'))
packer.addItem(Item(partno='Box-5',name='test',typeof='cube', WHD=(2, 1, 3), weight=1, level=1,loadbear=100, updown=True, color='olive'))

# calculate packing 
packer.pack(
    bigger_first=True,
    distribute_items=False,
    fix_point=True,
    check_stable=True,
    support_surface_ratio=0.75,
    number_of_decimals=0
)

In [8]:
box._plot()